# Demo

This notebook demos:
- The easiest and fastest way to get started with SimpleKGPipeline
- Controlling the schema to improve the Entity and Relation extraction
- Updating the Entity and Relation Extraction to domain specific needs

## Initial setup

Reading settings from `.env` file.

In [ ]:
import os

from dotenv import load_dotenv
import neo4j
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings import OpenAIEmbeddings
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline

In [ ]:
load_dotenv()

In [ ]:
file_path = "./data/Climate change - Wikipedia long.pdf"

Instantiate required objects: neo4j driver, an LLM and an embedder (for chunk embeddings)

In [ ]:
driver = neo4j.GraphDatabase.driver(
    os.getenv("NEO4J_URI", "bolt://localhost:7687"),
    auth=(
        os.getenv("NEO4J_USERNAME", "neo4j"),
        os.getenv("NEO4J_PASSWORD", "neo4j")
    )
)

llm = OpenAILLM(
    model_name="gpt-4o",
    model_params={
        "temperature": 0,
        "response_format": {"type": "json_object"}
    }
)

embedder = OpenAIEmbeddings(
    model="text-embedding-3-small",
)

## Quick start: Run SimpleKGPipeline

In [ ]:
pipeline = SimpleKGPipeline(
    driver=driver,
    llm=llm,
    embedder=embedder,
)

In [ ]:
await pipeline.run_async(
    file_path=file_path,
    document_metadata={
        "source": "Wikipedia",
    }
);

## Control the schema

In [ ]:
driver.execute_query("MATCH (n) DETACH DELETE n");

In [ ]:
from neo4j_graphrag.experimental.components.pdf_loader import PdfLoader
loader = PdfLoader()
document = await loader.run(filepath=file_path)

In [ ]:
from neo4j_graphrag.experimental.components.schema import SchemaFromTextExtractor

In [ ]:
extractor = SchemaFromTextExtractor(
    llm=llm,
)
schema = await extractor.run(text=document.text)
schema

In [ ]:
from neo4j_graphrag.experimental.utils.schema import schema_visualization

In [ ]:
VG = schema_visualization(schema)
VG.render()

In [ ]:
schema.save("my_schema.json", overwrite=True)


Make some changes to the JSON file

In [ ]:
from neo4j_graphrag.experimental.components.schema import GraphSchema
new_schema = GraphSchema.from_file("refined_schema.json")

VG = schema_visualization(new_schema)
VG.render()

In [ ]:
pipeline = SimpleKGPipeline(
    driver=driver,
    llm=llm,
    embedder=embedder,
    schema=new_schema,
)
await pipeline.run_async(
    file_path=file_path,
    document_metadata={
        "source": "Wikipedia",
        "schema": "edited schema",
    }
);

## Update the Entity extraction prompt

In [ ]:
driver.execute_query("MATCH (n) DETACH DELETE n");

In [ ]:
from neo4j_graphrag.generation.prompts import ERExtractionTemplate
additional_guidance = """
NAMING RULES:
- Use canonical short names whenever possible:
  * GreenhouseGas: CO2, CH4, N2O, H2O
  * ClimateEvent: Global warming, Sea level rise, Ocean heat content rise
  * RenewableSource: Solar power, Wind power, Hydropower, Renewable energy
  * HumanActivity: Fossil fuel combustion, Deforestation, Agriculture
  * Country: use common country names (France, India, China, etc.)
  * Agreement: Paris Agreement, Kyoto Protocol
- Avoid synonyms or redundant variants (e.g., “carbon dioxide” → “CO2”).
- Keep capitalization consistent and concise.
"""
er_prompt = ERExtractionTemplate().template + additional_guidance
pipeline = SimpleKGPipeline(
    driver=driver,
    llm=llm,
    embedder=embedder,
    schema=new_schema,
    prompt_template=er_prompt,
)
await pipeline.run_async(
    file_path=file_path,
    document_metadata={
        "source": "Wikipedia",
    },
);